In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, adjusted_rand_score, classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

methods = ["sw", "nw", "lstm", "svm", "assam", "GMM", "hmm", "castle", "tfidf_lr", "word2vec"]


In [ ]:
labels = pd.DataFrame()
for fold in range(5):
    gt = pd.read_csv(f"../../data/classification/folds/outer_fold_{fold}_test_labels.csv")
    labels = pd.concat([labels, gt])

In [ ]:
result_table = pd.DataFrame(columns=["method", "fold", "accuracy", "f1", "fdr", "mdr"])


for method in methods:
    for fold in range(5):
        gt = pd.read_csv(f"../../data/classification/folds/outer_fold_{fold}_test_labels.csv")
        gt = gt["0"]
        pred = pd.read_csv(f"../../data/classification/folds/predictions/{method}_outer_fold_{fold}_predictions.csv")
        pred = pred["0"]
        accuracy = accuracy_score(gt, pred)
        accuracy = np.round(accuracy,5)
        f1 = f1_score(gt, pred, average="weighted")
        f1 = np.round(f1,5)
        e0 = np.sum((gt == -1) & (pred != -1))
        e1 = np.sum((gt == -1) & (pred == -1))
        fdr = np.round(e0/(e0+e1 + 1e-10),5)

        a0 = np.sum((gt != -1) & (pred == -1))
        a1 = np.sum((gt != -1) & (pred != -1))
        mdr = np.round(a0/(a0+a1), 5)
        result_table = result_table.append({"method": method, "fold": fold, "accuracy": accuracy, "f1": f1, "fdr": fdr, "mdr": mdr}, ignore_index=True)



In [ ]:
result_table

In [ ]:
mean_results_by_fold = result_table.groupby("method").mean()
std_results_by_fold = result_table.groupby("method").std()

In [ ]:
#mean_results_by_fold.reindex(methods).round(5).to_csv("temp.csv", sep=";")

In [ ]:
lower_95 = mean_results_by_fold - 1.96*std_results_by_fold/np.sqrt(5)
higher_95 = mean_results_by_fold + 1.96*std_results_by_fold/np.sqrt(5)

In [ ]:
representable_labels = ["MSW", "MNW", "LSTM", "SVM", "ASSAM", "GMM", "HMM", "CASTLE", "TF-IDF-LR", "DK-Word2Vec"]

In [ ]:
result_table["method" ]= result_table["method"].apply(lambda x: representable_labels[methods.index(x)])

In [ ]:
for metric in ["accuracy"]:
    plt.figure(figsize=(10, 8))  
    sns.set_palette("deep")
    sns.set_style("whitegrid")
    sns.set_context("paper")
    ax = sns.boxplot(x=result_table["method"],y=result_table[metric])
    ax.set_ylim([0,1])
    ax.set_xlabel("", fontsize=12)
    ax.set_ylabel(metric, fontsize=12)
    plt.show()

In [ ]:
online_flood_result_table = pd.DataFrame(columns=["method", "fold", "minute", "accuracy", "f1", "fdr", "mdr"])


for method in methods:
    for fold in range(5):
        gt = pd.read_csv(f"../../data/classification/folds/outer_fold_{fold}_test_labels.csv")
        gt = gt["0"]
        for minute in range(11,21):

            pred = pd.read_csv(f"../../data/classification/online_floods/predictions/{method}/" + \
                               f"{method}_outer_fold_{fold}_min_{minute}_predictions.csv")
            pred = pred["0"]
            accuracy = accuracy_score(gt, pred)
            accuracy = np.round(accuracy,5)
            f1 = f1_score(gt, pred, average="weighted")
            f1 = np.round(f1,5)
            e0 = np.sum((gt == -1) & (pred != -1))
            e1 = np.sum((gt == -1) & (pred == -1))
            fdr = np.round(e0/(e0+e1 + 1e-10),5)

            a0 = np.sum((gt != -1) & (pred == -1))
            a1 = np.sum((gt != -1) & (pred != -1))
            mdr = np.round(a0/(a0+a1), 5)
            online_flood_result_table = online_flood_result_table.append({"method": method, "fold": fold, "minute": minute, "accuracy": accuracy, "f1": f1, "fdr": fdr, "mdr": mdr}, ignore_index=True)



In [ ]:
online_flood_result_table["method" ]= online_flood_result_table["method"].apply(lambda x: representable_labels[methods.index(x)])

In [ ]:
online_flood_result_table

In [ ]:
mean_metrics_by_minute = online_flood_result_table.groupby(["method", "minute"]).mean()
mean_metrics_by_minute = mean_metrics_by_minute.reindex(representable_labels, level=0)

In [ ]:
ax = sns.lineplot(data=mean_metrics_by_minute.reset_index(), x="minute", y="accuracy", hue="method")
ax.set_ylim([0,1])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()
ax = sns.lineplot(data=mean_metrics_by_minute.reset_index(), x="minute", y="f1", hue="method")
ax.set_ylim([0,1])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()
ax = sns.lineplot(data=mean_metrics_by_minute.reset_index(), x="minute", y="fdr", hue="method")
ax.set_ylim([0,1])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()
ax = sns.lineplot(data=mean_metrics_by_minute.reset_index(), x="minute", y="mdr", hue="method")
ax.set_ylim([0,1])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# K-fold cv paired t-test between accuracies of methods
p_value_table = pd.DataFrame(columns=["method1", "method2", "p_value"])

from scipy.stats import ttest_rel
for i in range(len(methods)):
    for j in range(i+1, len(methods)):
        method1 = representable_labels[i]
        method2 = representable_labels[j]
        t, p = ttest_rel(result_table[result_table["method"] == method1]["accuracy"],
                            result_table[result_table["method"] == method2]["accuracy"])
        p_value_table = p_value_table.append({"method1": method1, "method2": method2, "p_value": p}, ignore_index=True)

# apply bonferroni correction
alpha = 0.05
bonferroni_alpha = alpha/len(p_value_table)


In [ ]:
bonferroni_alpha

In [ ]:
p_value_table["significant"] = p_value_table["p_value"] < bonferroni_alpha

In [ ]:
p_value_table

In [ ]:
p_value_table[p_value_table["significant"] == True]

In [ ]:
# Convert to triu matrix
p_value_matrix = p_value_table.pivot(index="method1", columns="method2", values="p_value")

In [ ]:
p_value_matrix

In [ ]:
p_value_matrix.to_csv("p_value_matrix.csv", sep=";")

In [ ]:
# For each class find mean accuracy
accuracy_per_class = pd.DataFrame(columns=["method","fold", "class", "accuracy"])


for method in methods:
    for fold in range(5):
        gt = pd.read_csv(f"../../data/classification/folds/outer_fold_{fold}_test_labels.csv")
        gt = gt["0"]
        pred = pd.read_csv(f"../../data/classification/folds/predictions/{method}_outer_fold_{fold}_predictions.csv")
        pred = pred["0"]
        for label in sorted(gt.unique()):
            label_indices = gt == label
            accuracy = accuracy_score(gt[label_indices], pred[label_indices])
            accuracy = np.round(accuracy,5)
            accuracy_per_class = accuracy_per_class.append({"method": method, "fold": fold, "class": label, "accuracy": accuracy}, ignore_index=True)

In [ ]:
acc_by_cm = accuracy_per_class.groupby(["class", "method"]).mean().reset_index()

In [ ]:
from collections import defaultdict

In [ ]:
# For each method get classes where the method gets top 3 accuracy
top_classes = defaultdict(list)
worst_classes = defaultdict(list)
top_methods = defaultdict(list)
worst_methods = defaultdict(list)
for c in acc_by_cm["class"].unique():
    sor_acc = acc_by_cm[acc_by_cm["class"] == c].sort_values("accuracy", ascending=False)
    worst = sor_acc.tail(1)["accuracy"].values
    best =  sor_acc.head(1)["accuracy"].values
    good = sor_acc[sor_acc["accuracy"].values > best - 0.1]
    bad = sor_acc[sor_acc["accuracy"].values < worst + 0.1]
    for _,g in good.iterrows():
        top_classes[g["method"]].append(c)
        top_methods[c].append(g["method"])
    for _,b in bad.iterrows():
        worst_classes[b["method"]].append(c)
        worst_methods[c].append(b["method"])

In [ ]:
worst_classes

In [ ]:
mean_acc_by_class = accuracy_per_class.groupby(["class"]).mean()

In [ ]:
ax = sns.barplot(x=mean_acc_by_class.index, y=mean_acc_by_class["accuracy"], edgecolor="black", linewidth=1.5)
ax.set_xlabel("root cause")

In [ ]:
# For each sample find mean accuracy
accuracy_per_sample = pd.DataFrame(columns=["method","fold", "sample", "correct", "predicted_class", "true_class"])
for method in methods:
    for fold in range(5):
        gt = pd.read_csv(f"../../data/classification/folds/outer_fold_{fold}_test_labels.csv")
        gt = gt["0"]
        pred = pd.read_csv(f"../../data/classification/folds/predictions/{method}_outer_fold_{fold}_predictions.csv")
        pred = pred["0"]
        for sample in range(len(gt)):
            if gt[sample] == pred[sample]:
                correct = 1
            else:
                correct = 0
            accuracy_per_sample = accuracy_per_sample.append({"method": method, "fold": fold, "sample": f"{fold}_{sample}", "correct": correct, "predicted_class": pred[sample], "true_class": gt[sample]}, ignore_index=True)

In [ ]:
by_sample = accuracy_per_sample.groupby(["sample"]).apply(lambda x: np.mean(x["correct"]))
by_sample[by_sample < 0.2]

In [ ]:
accuracy_per_sample[accuracy_per_sample["sample"] == "3_21"]

In [ ]:
# Plot Word2Vec embeddings
labels.set_index("Unnamed: 0").sort_index()

In [ ]:
alarm_floods = pd.read_csv('../../data/preprocessed/Crane_alarm_floods.csv')

In [ ]:
import sys
sys.path.append("../")

import methods.word2vec as w2v

In [ ]:
vocab = {str(alarm): i for i, alarm in enumerate(alarm_floods["alarmNumber"].unique())} 

In [ ]:
model = w2v.Word2Vec_Classifier(vocab, 50, 4, lambda_= 1e-08, distance_threshold=0.3)
model.fit(alarm_floods, labels, 50)

In [ ]:
#model.plot_embeddings(list(range(0,104)))
model.plot_embeddings(list(range(40,50)))